## PROJECT. Оценка эффективности новой фичи

ЗАДАНИЕ

- Посчитать показатели Retention Rate седьмого дня и ARPPU за два периода: до и после внедрения фичи (май 2019/июнь 2019).
- Сформировать выводы по анализу.

In [1]:
# импорт библиотек
%matplotlib inline
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras 

Oпределим количество зарегистрировавшихся пользователей за весь период (до и после релиза). Создадим функцию getRegisteredUsersCount(), в которой выполним SQL-запрос. Подсчитаем количество зарегистрировавшихся пользователей registered_users_count в разрезе registration_date — дней регистрации.

In [2]:
def getRegisteredUsersCount():
    query = '''SELECT r.registration_date, count(distinct(r.user_id)) as registered_users_count
    FROM case9.registration r
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

registered_users_count = pd.DataFrame(getRegisteredUsersCount())

In [3]:
# преобразуем колонку registration_date в тип datetime:
registered_users_count['registration_date'] = pd.to_datetime(registered_users_count['registration_date'])

Мы получили количество пользователей, зарегистрировавшихся в мае и июне. Но мы не знаем, сколько из них впоследствии возвращалось в сервис.

Для восполнения этого информационного пробела мы можем посчитать количество активных пользователей в разрезе когорт, причём в качестве когорты будет выступать дата регистрации. Таким образом, мы определим количество активных пользователей по дням в разрезе даты регистрации.

Сперва получим данные об активных пользователях из базы и подсчитаем по дням количество активных пользователей active_users_count с указанием даты регистрации registration_date.

In [4]:
def getActiveUsersCountWithCohorts():
    query = '''SELECT ac.activity_date, r.registration_date, count(distinct(ac.user_id)) as active_users_count
    FROM case9.user_activity ac
    LEFT JOIN case9.registration r ON r.user_id = ac.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

active_users_count_with_cohorts = pd.DataFrame(getActiveUsersCountWithCohorts())

In [5]:
#  Преобразуем колонки с датами в тип datetime.
for col in ['activity_date','registration_date']:
    active_users_count_with_cohorts[col] = pd.to_datetime(active_users_count_with_cohorts[col])

Теперь у нас есть две таблицы:

- Таблица с количеством пользователей, которые изначально были в когортах, — это датафрейм registered_users_count.
- Таблица с количеством активных пользователей по дням в каждой когорте — это датафрейм active_users_count_with_cohorts.

Чтобы рассчитать процент вернувшихся пользователей на определённый день, нам нужно сначала объединить две таблицы.

In [7]:
retention_table = active_users_count_with_cohorts.merge(registered_users_count,on=['registration_date'],how='left')

В датафрейме retention_table нам нужно определить, какое количество дней прошло с момента регистрации до определённого дня активности. 

In [8]:
retention_table['lifetime'] = retention_table['activity_date'] - retention_table['registration_date']
# Нам нужно получить целое количество дней, поэтому значения в колонке с типом timedelta мы разделим на константу np.timedelta64(1,'D'):
retention_table['lifetime'] = retention_table['lifetime']/np.timedelta64(1,'D')
retention_table['lifetime'] = retention_table['lifetime'].astype(int) # Приведем тип к целому числу
retention_table.head()

,activity_date,registration_date,active_users_count,registered_users_count,lifetime
0,2019-05-01,2019-05-01,2590,6516,0
1,2019-05-02,2019-05-01,2146,6516,1
2,2019-05-02,2019-05-02,2049,5309,0
3,2019-05-03,2019-05-01,1637,6516,2
4,2019-05-03,2019-05-02,1817,5309,1


Чтобы посчитать ARPU, нам не хватает информации о выручке по дням, разбитой на когорты (как мы делали с retention_table).

Напишем SQL-запрос, который позволит получить выручку по дням в разрезе когорт. Этот запрос обернём в функцию getRevenue(), после чего получим данные по запросу и запишем в датафрейм revenue.

In [9]:
def getRevenue():
    query = '''SELECT re.date, r.registration_date, count(distinct(re.user_id)) as users_count_with_revenue, sum(re.revenue) as revenue
    FROM case9.user_revenue re
    LEFT JOIN case9.registration r ON r.user_id = re.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

revenue = pd.DataFrame(getRevenue())

In [10]:
# так же преобразуем даты в формат времени
for col in ['date','registration_date']:
    revenue[col] = pd.to_datetime(revenue[col])

Следующий шаг — объединение данных из датафрейма revenue с данными из датафрейма retention_table. Но прежде в датафрейме revenue следует изменить название колонки date на activity_date. Делаем это с тем, чтобы в дальнейшем объединять датафреймы по одинаковому названию колонок.

In [11]:
revenue = revenue.rename(columns={'date':'activity_date'})
retention_table_with_revenue = retention_table.merge(revenue,on=['registration_date','activity_date'],how='left')
# Произведём замену пропущенных значений на 0 с помощью функции fillna()
for col in ['revenue','users_count_with_revenue']:
    retention_table_with_revenue[col] = retention_table_with_revenue[col].fillna(0)
# Преобразуем тип в колонке users_count_with_revenue в целочисленный
retention_table_with_revenue['users_count_with_revenue'] = retention_table_with_revenue['users_count_with_revenue'].astype(int)

retention_table_with_revenue.head()

,activity_date,registration_date,active_users_count,registered_users_count,lifetime,users_count_with_revenue,revenue
0,2019-05-01,2019-05-01,2590,6516,0,1672,2357.590
1,2019-05-02,2019-05-01,2146,6516,1,1420,514.679
2,2019-05-02,2019-05-02,2049,5309,0,1367,1629.140
3,2019-05-03,2019-05-01,1637,6516,2,1048,390.952
4,2019-05-03,2019-05-02,1817,5309,1,1164,533.608


Теперь всё готово для расчёта показателей.

Рассчитаем Retention Rate седьмого дня для пользователей, привлеченных до релиза (май 2019).

In [12]:
# Фильтруем таблицу по Retention 7 дня за май
retention_7_may_table = retention_table_with_revenue[retention_table_with_revenue['lifetime']==7][retention_table_with_revenue['registration_date']>="2019-05-01"][retention_table_with_revenue['registration_date']<"2019-06-01"]
# расчет для этих когорт
retention_7_may = retention_7_may_table['active_users_count'].sum() / retention_7_may_table['registered_users_count'].sum()
print(retention_7_may)

0.12933000957960866


C:\Users\galja\AppData\Local\Temp\ipykernel_13336\70511395.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  retention_7_may_table = retention_table_with_revenue[retention_table_with_revenue['lifetime']==7][retention_table_with_revenue['registration_date']>="2019-05-01"][retention_table_with_revenue['registration_date']<"2019-06-01"]


Рассчитаем Retention Rate седьмого дня для пользователей, привлеченных после релиза (июнь 2019).

In [13]:
retention_7_june_table = retention_table_with_revenue[retention_table_with_revenue['lifetime']==7][retention_table_with_revenue['registration_date']>="2019-06-01"][retention_table_with_revenue['registration_date']<"2019-07-01"]
retention_7_june = retention_7_june_table['active_users_count'].sum() / retention_7_june_table['registered_users_count'].sum()
print(retention_7_june)

0.12560867530346634


C:\Users\galja\AppData\Local\Temp\ipykernel_13336\390074408.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  retention_7_june_table = retention_table_with_revenue[retention_table_with_revenue['lifetime']==7][retention_table_with_revenue['registration_date']>="2019-06-01"][retention_table_with_revenue['registration_date']<"2019-07-01"]


- Retention 7 дня в мае составляет 12,93%.
- Retention 7 дня в июне составляет 12,56%.

Теперь посчитаем показатель ARPPU до и после релиза.

In [15]:
# до релиза
may_arppu = retention_table_with_revenue['revenue'][retention_table_with_revenue['registration_date']>="2019-05-01"][retention_table_with_revenue['registration_date']<"2019-06-01"].sum() / retention_table_with_revenue['users_count_with_revenue'][retention_table_with_revenue['registration_date']>="2019-05-01"][retention_table_with_revenue['registration_date']<"2019-06-01"].sum()
print(may_arppu)

0.3061246038517031


In [16]:
# после релиза
june_arppu = retention_table_with_revenue['revenue'][retention_table_with_revenue['registration_date']>="2019-06-01"][retention_table_with_revenue['registration_date']<"2019-07-01"].sum() / retention_table_with_revenue['users_count_with_revenue'][retention_table_with_revenue['registration_date']>="2019-06-01"][retention_table_with_revenue['registration_date']<"2019-07-01"].sum()
print(june_arppu)

0.39319397180047666


- В мае, до релиза, ARPPU = 0,30 у.е.
- В июне, после релиза, ARPPU = 0,39 у.е.

### Выводы

Retention 7 дня за июнь незначительно ниже, чем в период до релиза. 

Но ARPPU (средний доход с 1 платящего клиента) вырос относительно мая.

Возможно, на ARPPU влияют другие факторы. Однако, можно сказать, что новая фича не является неэффективной.